# 1. Setup

## 1.1 Installing Libraries

Reference: [Llama Index Installation and Setup](https://docs.llamaindex.ai/en/stable/getting_started/installation/)

In [ ]:
!pip install python-dotenv llama-index chromadb llama-index-vector-stores-chroma llama-index-retrievers-bm25 EbookLib html2text

## 1.2 Importing Libraries

In [75]:
import chromadb

from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, PromptTemplate, get_response_synthesizer
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.llms.openai import OpenAI
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core.prompts import SelectorPromptTemplate

from ebooklib import epub
import uuid
import os
from pathlib import Path
from dotenv import load_dotenv

## 1.3 Importing Environment Variables

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## 1.4 Setting up Embedding Model

In [3]:
embed_model = OpenAIEmbedding(api_key=OPENAI_API_KEY)

## 1.5 Setting up LLM

In [4]:
llm = OpenAI(api_key=OPENAI_API_KEY, model_name="gpt-4o-mini", temperature=0.1)

# 2. Setting up IndexedVectorStore

In [5]:
# The name "IndexedVectorStore" emphasizes that the class handles both the vector store and the index

class IndexedVectorStore:
    def __init__(self):
        self.db = chromadb.PersistentClient(path="./db")
        self.chroma_collection = self.db.get_or_create_collection("transcription_project")
        self.vector_store = ChromaVectorStore(chroma_collection=self.chroma_collection)
        self.index = VectorStoreIndex.from_vector_store(
            self.vector_store,
            embed_model=embed_model,
        )

    def add_documents(self, documents: list) -> None:
        # Add the documents to the LlamaIndex and persist them
        for document in documents:
            self.index.insert(document)
        self.index.storage_context.persist(persist_dir="./db")

In [6]:
vectorstore = IndexedVectorStore()

# 3. Loading Data from Directory using `SimpleDirectoryReader`

Reference: [Loaders](https://docs.llamaindex.ai/en/stable/understanding/loading/loading/)

Extracting Metadata Reference: [SimpleDirectoryReader](https://docs.llamaindex.ai/en/stable/module_guides/loading/simpledirectoryreader/)

We can specify a function that will read each file and extract metadata that gets attached to the resulting Document object for each file by passing the function as `file_metadata`

In [7]:
def extract_epub_metadata(book_path: str) -> dict:
    book_path = Path(book_path)
    if not book_path.exists():
        raise FileNotFoundError(f"EPUB file not found at path: {book_path}")
    book = epub.read_epub(str(book_path))

    return {
        "id": f"epub-{uuid.uuid4().hex}",
        "title": book.get_metadata("DC", "title")[0][0].rstrip(".epub") if book.get_metadata("DC", "title") else "N/A",
        "author": book.get_metadata("DC", "creator")[0][0] if book.get_metadata("DC", "creator") else "",
        "language": book.get_metadata("DC", "language")[0][0] if book.get_metadata("DC", "language") else "",
        "description": book.get_metadata("DC", "description")[0][0] if book.get_metadata("DC", "description") else "",
        "type": "epub",
        "embeddings": "openaiembeddings"
    }

In [8]:
documents = SimpleDirectoryReader(input_dir="./data", file_metadata=extract_epub_metadata).load_data()

e:\Capelin\llama-index-test\.venv\Lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
e:\Capelin\llama-index-test\.venv\Lib\site-packages\ebooklib\epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


In [9]:
print(f"Total Documents: {len(documents)}")

Total Documents: 1


In [10]:
print(documents[0].metadata)

{'id': 'epub-1b5ac705d1b54cf580ddb53e7356f398', 'title': "Theological Instructions (Amuzish-e Aqa'id)", 'author': 'Muhammad Taqi Misbah Yazdi', 'language': 'en', 'description': '', 'type': 'epub', 'embeddings': 'openaiembeddings'}


Loading a new book

In [15]:
new_book = SimpleDirectoryReader(input_files=["./data/give_and_take.epub"], file_metadata=extract_epub_metadata).load_data()
print(f"Metadata of first element: {new_book[0].metadata}")

# This way, we can load a new book and can use the same VectorStore object to add the new book to the index

Metadata of first element: {'id': 'epub-6d7d7179a80e414c9c18e6715cae457f', 'title': 'Give and Tak', 'author': 'Unknown', 'language': 'en', 'description': '', 'type': 'epub'}


# 4. Transforming

After the data is loaded, you then need to process and transform your data before putting it into a storage system. These transformations include chunking, extracting metadata, and embedding each chunk. This is necessary to make sure that the data can be retrieved, and used optimally by the LLM.

An `IngestionPipeline` uses a concept of Transformations that are applied to input data. These Transformations are applied to your input data

Reference: [IngestionPipeline](https://docs.llamaindex.ai/en/stable/module_guides/loading/ingestion_pipeline/)

In [13]:
chunk_size = 512
overlap_percentage = 0.25
chunk_overlap = int(chunk_size * overlap_percentage)

print(f"Chunk Size: {chunk_size}, Overlap Percentage: {overlap_percentage}, Chunk Overlap: {chunk_overlap}")

Chunk Size: 512, Overlap Percentage: 0.25, Chunk Overlap: 128


In [14]:
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap),
        embed_model # OpenAIEmbedding
    ],
    vector_store=vectorstore.vector_store,
)

In [15]:
documents = pipeline.run(documents=documents)

In [16]:
print(f"Total Documents: {len(documents)}")

Total Documents: 513


Function to load and add a new book to vectorstore

In [18]:
def add_book(book_path: str):
    print(f"Loading book from path: {book_path}")
    new_book = SimpleDirectoryReader(input_files=[book_path], file_metadata=extract_epub_metadata).load_data()
    print(f"Loaded book with metadata: {new_book[0].metadata}")
    new_book = pipeline.run(documents=new_book)
    print("Book added successfully!")

In [19]:
add_book("./data/give_and_take.epub")

Loading book from path: ./data/give_and_take.epub


e:\Capelin\llama-index-test\.venv\Lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
e:\Capelin\llama-index-test\.venv\Lib\site-packages\ebooklib\epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


Loaded book with metadata: {'id': 'epub-b09eb4f758f54495a29b6b23763dffb3', 'title': 'Give and Tak', 'author': 'Unknown', 'language': 'en', 'description': '', 'type': 'epub'}
Book added successfully!


# 5. Query Translation

In [24]:
question_template = """You are an AI language model assistant specializing in query expansion. Your task is to generate {num_queries} diverse versions of the given user question. These variations will be used to retrieve relevant documents from a vector database, helping to overcome limitations of distance-based similarity search.

Original question: {query}

Instructions:
1. Create {num_queries} unique variations of the original question.
2. Ensure each variation maintains the core intent of the original question.
3. Use different phrasings, synonyms, or perspectives for each variation.
4. Consider potential context or implications not explicitly stated in the original question.
5. Avoid introducing new topics or drastically changing the meaning of the question.

Please provide your {num_queries} question variations, each on a new line:
"""

question_prompt = PromptTemplate(question_template)

In [28]:
def generate_query_variations(question: str, num_queries: int):
    print(f"Generating query variations for: {question}")

    fmt_prompt = question_prompt.format(num_queries=num_queries, query=question)
    response = llm.complete(fmt_prompt)
    queries = response.text.split("\n")

    print("Generated query variations:")
    for query in queries:
        print(f"  {query}")

    return queries

In [29]:
question = "Why is there only one God?"
num_queries = 5
query_variations = generate_query_variations(question, num_queries)

Generating query variations for: Why is there only one God?
Generated query variations:
  1. What is the rationale behind the belief in a singular God?
  2. How do monotheistic religions justify the existence of only one deity?
  3. What factors contribute to the concept of a solitary divine being?
  4. Is there a specific reason for the monotheistic view of a singular God?
  5. What leads to the idea that there can be only one supreme being in various faiths?


# 6. Performing Vector Search

In [71]:
top_n = 5

vector_retriever = vectorstore.index.as_retriever(similarity_top_k=top_n)

In [72]:
retriever = QueryFusionRetriever(
    [vector_retriever],
    similarity_top_k=top_n,
    num_queries=num_queries,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    # query_gen_prompt="...",  # we could override the query generation prompt here
)

In [73]:
import nest_asyncio

nest_asyncio.apply()

In [74]:
nodes_with_scores = retriever.retrieve(question)

Generated queries:
1. What are the reasons for monotheism in various religions?
2. How do different cultures and belief systems explain the concept of a single deity?
3. Are there any philosophical arguments for the existence of a singular supreme being?
4. How does the concept of one God differ across different religious traditions?


In [34]:
# Printing first document
print("------- BOOK INFO -------")
print(f"Book Title: {nodes_with_scores[0].metadata['title']}")
print(f"Book ID   : {nodes_with_scores[0].metadata['id']}")
print(f"Author    : {nodes_with_scores[0].metadata['author']}")

print("\n------- TEXT -------")
print(nodes_with_scores[0].text)

------- BOOK INFO -------
Book Title: Theological Instructions (Amuzish-e Aqa'id)
Book ID   : epub-1b5ac705d1b54cf580ddb53e7356f398
Author    : Muhammad Taqi Misbah Yazdi

------- TEXT -------
By explaining the
contradiction in this belief, it is possible to nullify the argument of those
who held such a view.

In order to establish the oneness of God, the Supreme, many arguments have
been demonstrated in the different books of theology and philosophy. Here we
are going to demonstrate an argument, which encompasses the oneness of
lordship and rejects the polytheistic beliefs.

## Proofs For The Oneness Of God

The assumption that the universe has two or more gods can solely be imagined
through a few possibilities:

Firstly, it can be considered that every phenomenon of the universe is created
and is an effect of all the assumed gods. The second assumption could be that
each particular group of phenomena is an effect (or created by a particular
god) of one of the assumed gods. Finally, t

# 7. Generate Answer

In [77]:
answer_template = """You are a knowledgeable AI assistant tasked with answering questions based on the provided context. Your goal is to provide a comprehensive, accurate, and well-structured response using Chain-of-Thought reasoning.

Context:
{context_str}

Question: {query_str}

Instructions:
1. Carefully analyze the given context and question.
2. Use Chain-of-Thought reasoning to break down your answer into clear steps:
   a. First, identify the key components of the question, such as sub-problems that need to be explained before an answer can be derived
   b. Then, for each component, explain your thought process as you analyze the relevant information from the context.
   c. Show how you're connecting different pieces of information to form your conclusion.
3. Provide a detailed answer using only the information from the context.
4. If the context doesn't contain enough information to fully answer the question, state this clearly and explain why.
5. Organize your response with appropriate headings and subheadings for clarity.
6. Use bullet points or numbered lists where applicable to improve readability.
7. If relevant, include brief examples or analogies to illustrate key points.
8. After your detailed Chain-of-Thought reasoning, summarize your main points at the end of the response.
9. At the end, list all the contexts used in your reasoning. After your response, add a "References" section where you list the full contexts that you used arrive at your answer. Provide as much detail as available from each context (e.g., book title, author, full text of the relevant contexts. For video sources, include the url to the video.

For the references, use the format:

# References:
(for each context:)
## Context Id: title
Context excerpt (print as it is)

Please format your entire response in markdown for optimal readability.
"""

answer_prompt = PromptTemplate(answer_template)
answer_prompt_sel = SelectorPromptTemplate(answer_prompt)

## Response Modes

`REFINE`:

Refine is an iterative way of generating a response. We first use the context in the first node, along with the query, to generate an initial answer. We then pass this answer, the query, and the context of the second node as input into a “refine prompt” to generate a refined answer. We refine through N-1 nodes, where N is the total number of nodes.


`COMPACT`:

Compact and refine mode first combine text chunks into larger consolidated chunks that more fully utilize the available context window, then refine answers across them. This mode is faster than refine since we make fewer calls to the LLM.


`SIMPLE_SUMMARIZE`:

Merge all text chunks into one, and make a LLM call. This will fail if the merged text chunk exceeds the context window size.


`TREE_SUMMARIZE`:

Build a tree index over the set of candidate nodes, with a summary prompt seeded with the query. The tree is built in a bottoms-up fashion, and in the end the root node is returned as the response


`GENERATION`:

Ignore context, just use LLM to generate a response.

`NO_TEXT`:

Return the retrieved context nodes, without synthesizing a final response.

`CONTEXT_ONLY`:

Returns a concatenated string of all text chunks.

`ACCUMULATE`:

Synthesize a response for each text chunk, and then return the concatenation.

`COMPACT_ACCUMULATE`:

Compact and accumulate mode first combine text chunks into larger consolidated chunks that more fully utilize the available context window, then accumulate answers for each of them and finally return the concatenation. This mode is faster than accumulate since we make fewer calls to the LLM.


In [61]:
def generate_answer(question, documents, answer_prompt):
    response_synthesizer = get_response_synthesizer(response_mode="compact", llm=llm, text_qa_template=answer_prompt)

    response = response_synthesizer.synthesize(
        question, nodes=documents
    )

    return response

In [62]:
answer_md = generate_answer(question, nodes_with_scores, answer_prompt_sel)

In [63]:
print(answer_md)

# Answer:

## Key Components:
1. Understanding the concept of monotheism and the rejection of polytheism.
2. Exploring the arguments against the existence of multiple gods.
3. Analyzing the relationship between creatorship, lordship, and the oneness of God.
4. Examining the illusion of several gods and how it can be nullified.

## Chain-of-Thought Reasoning:

### 1. Monotheism vs. Polytheism:
- The context emphasizes the importance of establishing the oneness of God to reject polytheistic beliefs.
- Polytheism arises from various factors like idol worship, human tendencies, and the influence of tyrants.
- Monotheism asserts the belief in one God as the sole creator and lord of the universe.

### 2. Arguments Against Multiple Gods:
- The text presents arguments against the existence of multiple gods creating the universe.
- It refutes the idea of each phenomenon having several gods, as it would lead to a multitude of existents, contrary to the observed unity in creation.
- The complexit